In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
import copy

from typing import List, Optional, Tuple
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

from torchdiffeq import odeint
import torch.optim as optim

import time
import os
import shutil
import datetime
import sys
sys.path.append('..')


device = 'cpu'

In [2]:

from src.preprocessing import *
from src.NeuralODE_model import NeuralODE


In [3]:
# Загружаем обученный мост и train-test датасеты
x1_train, x1_test, x0_test, model_bridge, cfg_bridge = get_bridge('gaussian_dim-5_mean-3_var-1')

    # Инициализация NeuralODE
NeuralODE_model = NeuralODE(dim=cfg_bridge.dim, m=64) 

# Генерация траеткторий для обучения
dataloader, tensor_train = get_traj_dataloader(model_bridge, x1_train, num_steps=20, cnt_samples=10)

loss_list = []
dim = tensor_train.shape[2]

# Определение целевых показателей
# train
train_target_mean = tensor_train[:,-1,:].mean(0).mean(0)
train_target_cov = torch.cov(torch.cat([tensor_train[:,0,:], tensor_train[:,-1,:]], dim=1).T)[dim:, :dim].diag().mean(0)
train_target_var = tensor_train[:,-1,:].var(dim=0).mean(0)

train_optimal_result_dict = {'mean':train_target_mean.item(), 'var':  train_target_var.item(), 'cov': train_target_cov.item()}
result_list_train = {k: [] for k in train_optimal_result_dict.keys()}

# test
test_optimal_result_dict = {'mean':train_target_mean.item(), 'var':  train_target_var.item(), 'cov': train_target_cov}
result_list_test = {k: [] for k in test_optimal_result_dict.keys()}

print('Характеристики моста:')
print('mean: ', x1_train.mean(0).mean(0).item(), '->', train_target_mean.item())
print('var: ', x1_train.var(0).mean(0).item(), '->', train_target_var.item())
print('cov: ', train_target_cov.item())

Seed set to 42


Характеристики моста:
mean:  3.001769542694092 -> -2.890745162963867
var:  1.0090525150299072 -> 1.0331761837005615
cov:  0.6302254796028137


In [4]:
# Фоомирование директории для сохранения результатов
RESULT_DIR = '../results/' + cfg_bridge.paths.experiments_dir_name + '/'
if os.path.exists(RESULT_DIR):
    shutil.rmtree(RESULT_DIR)
os.makedirs(RESULT_DIR, exist_ok=True)

OmegaConf.save(cfg_bridge, RESULT_DIR + 'config_bridge.yaml')

In [12]:
# Train-loop
num_epochs = 5

optimizer =optim.RMSprop(NeuralODE_model.parameters(), lr=1e-5, weight_decay=0)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5)


for epoch in range(num_epochs):
    NeuralODE_model.train()
    total_loss = 0.0
    with tqdm(total=len(dataloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for batch in dataloader:
            optimizer.zero_grad()  # Обнуляем градиенты
            
            # Получаем текущую обучающую пару
            y0 = batch[0][:, 0, :]  # Начальное состояние
            t = torch.linspace(0, 1, steps=21)  # Временные точки
    
            # Прогон
            pred = odeint(NeuralODE_model, y0, t)
            pred = pred.permute(1, 0, 2)  # Согласование размерности

            # Вычисление функции потерь
            # MAE по траекторям (точкам)
            loss_base = torch.mean(torch.abs(pred - batch[0]))

            # MAE по дисперсии конечного распределения
            pred_final_var = pred[:, -1, :].var(dim=0).mean(0)
            loss_var = 1*torch.abs(pred_final_var - train_target_var)
            
            # MAE по кросс-ковариации
            dim = pred.shape[2]
            initial_state = pred[:, 0, :] 
            final_state = pred[:, -1, :]
            combined = torch.cat([initial_state, final_state], dim=1)
            cov_matrix = torch.cov(combined.T)
            cov_pred = cov_matrix[:dim, dim:].diag().mean()
            
            loss_cov = 3*torch.abs(cov_pred - train_target_cov)

            # Полная ошибка
            loss = (loss_base + loss_var + loss_cov)
            
            loss.backward()
            optimizer.step()
            
            loss_list.append(loss.item())
            total_loss += loss.item()

            pbar.update(1)  # Увеличиваем прогресс-бар на 1
            
    epoch_loss = total_loss / len(dataloader)
    scheduler.step(epoch_loss)

    # Расчет метрик качества на train и test
    with torch.no_grad():
        # на train
        pred_y = odeint(NeuralODE_model, x1_train, t)
        result_list_train['mean'].append(pred_y[-1].mean(0).mean(0).item())
        result_list_train['var'].append(pred_y[-1].var(0).mean(0).item())
        result_list_train['cov'].append(torch.cov(torch.cat([pred_y[0], pred_y[-1]], dim=1).T)[dim:, :dim].diag().mean(0).item())

        # на test
        pred_y = odeint(NeuralODE_model, x1_test, t)
        result_list_test['mean'].append(pred_y[-1].mean(0).mean(0).item())
        result_list_test['var'].append(pred_y[-1].var(0).mean(0).item())
        result_list_test['cov'].append(torch.cov(torch.cat([pred_y[0], pred_y[-1]], dim=1).T)[dim:, :dim].diag().mean(0).item())

        # Печать результатов
        for i, k in enumerate(result_list_train.keys()):
            plt.plot(result_list_train[k])
            plt.plot(np.arange(len(result_list_train[k])), train_optimal_result_dict[k] * np.ones(len(result_list_train[k])), label="optimal", linestyle="--")
            plt.title(k.capitalize())
            if i == 0:
                plt.legend()
            plt.savefig(RESULT_DIR + f"/train_convergence_{k}.png")
            plt.close()
    
            # test
        for i, k in enumerate(result_list_test.keys()):
            plt.plot(result_list_test[k])
            plt.plot(np.arange(len(result_list_test[k])), test_optimal_result_dict[k] * np.ones(len(result_list_test[k])), label="optimal", linestyle="--")
            plt.title(k.capitalize())
            if i == 0:
                plt.legend()
            plt.savefig(RESULT_DIR + f"/test_convergence_{k}.png")
            plt.close()

# Сохраняем результаты
torch.save(NeuralODE_model.state_dict(), RESULT_DIR + "NeuralODE_model.pt")

df_result = pd.DataFrame(result_list_train)
df_result.to_csv(RESULT_DIR + 'df_result_train.csv')
df_result.to_pickle(RESULT_DIR+ 'df_result_train.pkl')

df_result = pd.DataFrame(result_list_test)
df_result.to_csv(RESULT_DIR + 'result_list_test.csv')
df_result.to_pickle(RESULT_DIR+ 'result_list_test.pkl')

Epoch 1/5: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:09<00:00,  7.99batch/s]
C:\Users\Ксения Николаевна\AppData\Local\Temp\ipykernel_13196\200628681.py:82: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  plt.plot(np.arange(len(result_list_test[k])), test_optimal_result_dict[k] * np.ones(len(result_list_test[k])), label="optimal", linestyle="--")
Epoch 5/5: 100%|████████████████████████████████████████████████████████████████████| 79/79 [00:09<00:00,  8.21batch/s]
